In [1]:
# 1. 导入需要的库
import akshare as ak  # akshare 是核心的数据获取库
import pandas as pd    # pandas 用于数据处理和保存到Excel

print("开始获取可转债数据，请稍候...")

try:
    # 2. 调用接口获取可转债数据
    # ak.bond_zh_cov() 会返回一个pandas的DataFrame对象，这是一种表格形式的数据结构。
    bond_zh_cov_df = ak.bond_zh_cov()

    # 3. 定义你想要保存的文件名
    file_name = "可转债数据清单.xlsx"

    # 4. 将数据保存到Excel文件
    # 我们使用 to_excel 方法。其中：
    # - excel_writer: 指定了要保存的文件路径和名称。
    # - index=False: 这是一个很重要的参数。如果不设置，pandas会默认把数据行的索引（0, 1, 2, ...）也写进Excel的第一列，通常我们不希望这样，所以设为False。
    # - engine='openpyxl': 明确指定使用 openpyxl 引擎来写入xlsx文件。
    bond_zh_cov_df.to_excel(excel_writer=file_name, index=False, engine='openpyxl')

    # 5. 打印成功信息
    # 给出一个明确的提示，告知用户文件已经成功生成以及它的名字。
    print(f"数据已成功获取并保存到文件: {file_name}")
    print(f"共获取到 {len(bond_zh_cov_df)} 条数据。")

except Exception as e:
    # 6. 异常处理
    # 如果在获取或保存数据的过程中发生任何错误，打印出错误信息，方便排查问题。
    print(f"处理过程中发生错误: {e}")

开始获取可转债数据，请稍候...


  0%|          | 0/2 [00:00<?, ?it/s]

c:\Program Files\Python313\Lib\site-packages\akshare\bond\bond_zh_cov.py:342: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  big_df = pd.concat(objs=[big_df, temp_df], ignore_index=True)


数据已成功获取并保存到文件: 可转债数据清单.xlsx
共获取到 983 条数据。


In [2]:
# 1. 导入我们需要的库
import akshare as ak  # akshare 是获取数据的核心
import pandas as pd    # pandas 用于数据处理和与Excel交互

print("正在从同花顺获取可转债数据，这可能需要一点时间...")

try:
    # 2. 调用同花顺的接口获取可转债数据
    # 这次我们使用的是 bond_zh_cov_info_ths 函数
    bond_data_df = ak.bond_zh_cov_info_ths()

    # 3. 为文件命名，最好能区分出来源
    file_name = "可转债数据清单_同花顺.xlsx"

    # 4. 将数据写入Excel文件
    # 同样，我们使用 to_excel 方法，并且不保存pandas的索引列
    bond_data_df.to_excel(excel_writer=file_name, index=False, engine='openpyxl')

    # 5. 任务完成，给出一个清晰的反馈
    print(f"数据成功保存至文件: {file_name}")
    print(f"本次共获取 {len(bond_data_df)} 条可转债数据。")

except Exception as e:
    # 6. 如果过程中出现任何问题，友好地提示错误
    print(f"在处理过程中发生了错误: {e}")

正在从同花顺获取可转债数据，这可能需要一点时间...
数据成功保存至文件: 可转债数据清单_同花顺.xlsx
本次共获取 890 条可转债数据。


In [1]:
# 1. 导入必要的库
import akshare as ak
import pandas as pd

print("任务开始：准备合并与对比东方财富和同花顺的可转债数据...")

try:
    # 2. 分别获取两份数据
    print("步骤 1/5: 正在获取东方财富数据...")
    em_df = ak.bond_zh_cov()
    print("步骤 2/5: 正在获取同花顺数据...")
    ths_df = ak.bond_zh_cov_info_ths()

    # 3. 为两份数据的列名添加来源前缀
    # 这是关键一步，确保合并后能清晰地区分每个字段的来源
    em_df_prefixed = em_df.add_prefix('东方财富_')
    ths_df_prefixed = ths_df.add_prefix('同花顺_')
    print("步骤 3/5: 已为数据列名添加来源前缀。")

    # 4. 执行数据合并
    # 我们使用 'outer' 合并策略，确保任何一方独有的数据都不会丢失。
    # 合并的关键键是各自的'债券代码'。
    # indicator=True 会自动生成一个名为 '_merge' 的列，告诉我们每一行的来源状态。
    merged_df = pd.merge(
        em_df_prefixed,
        ths_df_prefixed,
        left_on='东方财富_债券代码',
        right_on='同花顺_债券代码',
        how='outer',
        indicator=True
    )
    print("步骤 4/5: 数据合并完成。")

    # 5. 根据合并结果创建来源标识列
    # 将 '_merge' 列的专业术语（left_only, right_only, both）转换成你想要的中文描述。
    status_map = {
        'left_only': '仅东方财富',
        'right_only': '仅同花顺',
        'both': '同时存在'
    }
    merged_df['数据来源'] = merged_df['_merge'].map(status_map)
    merged_df = merged_df.drop(columns=['_merge']) # 用完后就删掉辅助列

    # 6. 精心重排列表，实现“近似列并列”的效果
    # 这部分是实现你核心需求的魔法。我们手动定义一个理想的列顺序。
    # 首先，定义好我们希望并排对比的列
    paired_columns = [
        '东方财富_债券代码', '同花顺_债券代码',
        '东方财富_债券简称', '同花顺_债券简称',
        '东方财富_上市时间', '同花顺_上市日期',
        '东方财富_申购日期', '同花顺_申购日期',
        '东方财富_申购代码', '同花顺_申购代码',
        '东方财富_正股代码', '同花顺_正股代码',
        '东方财富_正股简称', '同花顺_正股简称',
        '东方财富_转股价', '同花顺_转股价格',
        '东方财富_中签率', '同花顺_中签率',
        '东方财富_中签号发布日', '同花顺_中签公布日',
        '东方财富_原股东配售-每股配售额', '同花顺_每股获配额',
    ]

    # 然后，找出那些只在某一个数据源存在的列
    all_current_columns = merged_df.columns.tolist()
    unique_columns = [col for col in all_current_columns if col not in paired_columns and col != '数据来源']
    
    # 最终的列顺序：并列对比的列 + 独有的列 + 我们创建的来源标识列
    final_column_order = paired_columns + unique_columns + ['数据来源']
    
    # 应用这个顺序
    final_df = merged_df[final_column_order]

    # 7. 保存到Excel
    file_name = "可转债数据对比表.xlsx"
    final_df.to_excel(file_name, index=False, engine='openpyxl')
    
    print(f"步骤 5/5: 处理完成！")
    print(f"合并后的数据已保存至文件: {file_name}")
    print("文件中包含了并列对比的列和来源标识，方便您进行下一步分析。")

except Exception as e:
    print(f"处理过程中发生错误: {e}")

任务开始：准备合并与对比东方财富和同花顺的可转债数据...
步骤 1/5: 正在获取东方财富数据...


  0%|          | 0/2 [00:00<?, ?it/s]

c:\Program Files\Python313\Lib\site-packages\akshare\bond\bond_zh_cov.py:342: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  big_df = pd.concat(objs=[big_df, temp_df], ignore_index=True)


步骤 2/5: 正在获取同花顺数据...
步骤 3/5: 已为数据列名添加来源前缀。
步骤 4/5: 数据合并完成。
步骤 5/5: 处理完成！
合并后的数据已保存至文件: 可转债数据对比表.xlsx
文件中包含了并列对比的列和来源标识，方便您进行下一步分析。


In [8]:
import akshare as ak
import pandas as pd
import numpy as np

print("启动最终版数据清洗任务（已修正过滤规则）...")

try:
    # === 步骤 1: 获取并合并原始数据 ===
    print("正在获取东方财富和同花顺数据...")
    em_df = ak.bond_zh_cov()
    ths_df = ak.bond_zh_cov_info_ths()

    em_df_prefixed = em_df.add_prefix('东方财富_')
    ths_df_prefixed = ths_df.add_prefix('同花顺_')

    merged_df = pd.merge(
        em_df_prefixed,
        ths_df_prefixed,
        left_on='东方财富_债券代码',
        right_on='同花顺_债券代码',
        how='outer',
        indicator=True
    )
    
    status_map = {'left_only': '仅东方财富', 'right_only': '仅同花顺', 'both': '同时存在'}
    merged_df['数据来源'] = merged_df['_merge'].map(status_map)
    merged_df.drop(columns=['_merge'], inplace=True)
    print("数据获取与初步合并完成。")

    # === 步骤 2: **修正并执行**精准过滤 ===
    print("正在执行精准过滤，分离可转债与非可转债记录...")
    initial_rows = len(merged_df)

    # **核心修正**: 使用 str.contains() 来实现更灵活的匹配
    # 条件：简称中包含“债”，但**不包含**“转债”
    is_non_convertible_bond = (merged_df['东方财富_债券简称'].str.contains('债', na=False)) & \
                              (~merged_df['东方财富_债券简称'].str.contains('转债', na=False))

    # 根据这个更精准的规则来分离数据
    filtered_out_df = merged_df[is_non_convertible_bond].copy()
    main_df = merged_df[~is_non_convertible_bond].copy()
    
    # === 步骤 3: 保存被过滤掉的非可转债记录 ===
    removed_file_name = "非可转债记录清单.xlsx"
    filtered_out_df.to_excel(removed_file_name, index=False, engine='openpyxl')
    
    print(f"过滤完成。总计 {initial_rows} 条，保留 {len(main_df)} 条进行下一步处理，剔除 {len(filtered_out_df)} 条非可转债记录。")
    print(f"被剔除的记录已保存至: {removed_file_name}")

    # === 步骤 4: 在核心数据中“标记”退市状态 ===
    print("正在为可转债数据添加'债券状态'标记...")
    main_df['债券状态'] = '正常'
    main_df.loc[main_df['东方财富_债券简称'].str.contains('退', na=False), '债券状态'] = '已退市'

    # === 步骤 5: 对保留的全部可转债数据进行深度清洗 ===
    print("正在执行各批次数据清洗...")
    # --- 第一批：统一正股简称 ---
    condition = (main_df['数据来源'] == '同时存在') & \
                (main_df['东方财富_正股简称'].notna()) & (main_df['同花顺_正股简称'].notna()) & \
                (main_df['东方财富_正股简称'] != main_df['同花顺_正股简称'])
    main_df.loc[condition, '东方财富_正股简称'] = main_df.loc[condition, '同花顺_正股简称']
    
    # --- 第二批：标准化“中签率”单位 ---
    main_df['东方财富_中签率'] = pd.to_numeric(main_df['东方财富_中签率'], errors='coerce')
    main_df['同花顺_中签率'] = pd.to_numeric(main_df['同花顺_中签率'], errors='coerce')
    cond_em_large = main_df['东方财富_中签率'] > (main_df['同花顺_中签率'] * 99)
    main_df.loc[cond_em_large, '东方财富_中签率'] /= 100

    # --- 第三批：清理并统一“中签号”格式 ---
    main_df['同花顺_中签号'] = main_df['同花顺_中签号'].astype(str).str.replace(r'[；、\s\n]+', ',', regex=True).str.strip(',')

    # --- 第四批：补充关键字段的缺失值 ---
    paired_columns_map = {
        '东方财富_上市时间': '同花顺_上市日期', '东方财富_申购日期': '同花顺_申购日期',
        '东方财富_转股价': '同花顺_转股价格', '东方财富_中签率': '同花顺_中签率'
    }
    for left_col, right_col in paired_columns_map.items():
        main_df[left_col].fillna(main_df[right_col], inplace=True)
        main_df[right_col].fillna(main_df[left_col], inplace=True)
    
    print("所有清洗批次执行完毕。")

    # === 步骤 6: 重排列表并保存最终的回测专用数据集 ===
    status_col = ['债券状态']
    paired_columns_order = [
        '东方财富_债券代码', '同花顺_债券代码', '东方财富_债券简称', '同花顺_债券简称',
        '东方财富_上市时间', '同花顺_上市日期', '东方财富_申购日期', '同花顺_申购日期',
        '东方财富_申购代码', '同花顺_申购代码', '东方财富_正股代码', '同花顺_正股代码',
        '东方财富_正股简称', '同花顺_正股简称', '东方财富_转股价', '同花顺_转股价格',
        '东方财富_中签率', '同花顺_中签率'
    ]
    all_current_columns = main_df.columns.tolist()
    unique_columns = [col for col in all_current_columns if col not in (paired_columns_order + status_col + ['数据来源'])]
    final_column_order = status_col + paired_columns_order + unique_columns + ['数据来源']
    final_df = main_df[final_column_order]

    cleaned_file_name = "可转债全量数据_清洗后(含退市).xlsx"
    final_df.to_excel(cleaned_file_name, index=False, engine='openpyxl')
    
    print("\n任务成功完成！")
    print(f"专为回测优化的全量数据集已保存至: {cleaned_file_name}")

except Exception as e:
    print(f"\n处理过程中发生错误: {e}")

启动最终版数据清洗任务（已修正过滤规则）...
正在获取东方财富和同花顺数据...


  0%|          | 0/2 [00:00<?, ?it/s]

c:\Program Files\Python313\Lib\site-packages\akshare\bond\bond_zh_cov.py:342: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  big_df = pd.concat(objs=[big_df, temp_df], ignore_index=True)


数据获取与初步合并完成。
正在执行精准过滤，分离可转债与非可转债记录...
过滤完成。总计 984 条，保留 964 条进行下一步处理，剔除 20 条非可转债记录。
被剔除的记录已保存至: 非可转债记录清单.xlsx
正在为可转债数据添加'债券状态'标记...
正在执行各批次数据清洗...
所有清洗批次执行完毕。


C:\Users\Administrator\AppData\Local\Temp\1\ipykernel_2940\3774583776.py:78: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  main_df[left_col].fillna(main_df[right_col], inplace=True)
C:\Users\Administrator\AppData\Local\Temp\1\ipykernel_2940\3774583776.py:79: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values alwa


任务成功完成！
专为回测优化的全量数据集已保存至: 可转债全量数据_清洗后(含退市).xlsx
